In [ ]:
import h5py
import numpy as np
import os,random
from keras.layers import Input,Reshape,ZeroPadding2D,Conv2D,Dropout,Flatten,Dense,Activation,MaxPooling2D,AlphaDropout,Conv1D,MaxPooling1D
from keras import layers
import keras.models as Model
from keras.regularizers import *
from keras.optimizers import adam
import seaborn as sns
import keras
import keras.backend.tensorflow_backend as tfback
import tensorflow as tf
import matplotlib.pyplot as plt
import gc
%matplotlib inline
os.environ["KERAS_BACKEND"] = "tensorflow"
import pandas as pd

In [ ]:
def _get_available_gpus():
    """Get a list of available gpu devices (formatted as strings).

    # Returns
        A list of available GPU devices.
    """
    #global _LOCAL_DEVICES
    if tfback._LOCAL_DEVICES is None:
        devices = tf.config.list_logical_devices()
        tfback._LOCAL_DEVICES = [x.name for x in devices]
    return [x for x in tfback._LOCAL_DEVICES if 'device:gpu' in x.lower()]


In [ ]:
tfback._get_available_gpus = _get_available_gpus
tfback._get_available_gpus()

In [ ]:
#tf.config.experimental_list_devices()
tf.config.list_logical_devices()

In [ ]:
#loading data
f = h5py.File("../input/dataset/data.hdf5", 'r')
X_train = np.array(f['train'])
# X_train = np.vstack((X_train, X_big_train))
# X_train = torch.from_numpy(X_train).reshape(X_train.shape[0], 1,X_train.shape[1], X_train.shape[2])
# X_train = X_train.reshape(X_train.shape[0],X_train.shape[2], X_train.shape[1])
print(X_train.shape)

X_test = np.array(f['test'])
# X_test = torch.from_numpy(X_test).reshape(X_test.shape[0], 1,X_test.shape[1], X_test.shape[2])

Y_raw_train = pd.read_csv('../input/dataset/train_labels.csv').to_numpy()
# Y_train = np.vstack((Y_train, Y_big_train))

In [ ]:
class_index = {
 'FM': 0,
 'OQPSK':1,
 'BPSK':2,
 '8PSK':3,
 'AM-SSB-SC':4,
 '4ASK':5,
 '16PSK':6,
 'AM-DSB-SC':7, 
 'QPSK': 8, 
 'OOK':9,
    3:0,
    6:1,
    8:2,
    9:3,
    10:4,
    11:5,
    12:6,
    16:7,
    19:8,
    22:9
}
index_class = {
 0:'FM',
 1:'OQPSK',
 2:'BPSK',
 3:'8PSK',
 4:'AM-SSB-SC',
 5:'4ASK',
 6:'16PSK',
 7:'AM-DSB-SC', 
 8:'QPSK', 
 9:'OOK',
}

classnum = 10
def classToIndex(catg):
    return class_index[catg]

#direct mapping, not one-hot
def setToNum(dataset):
    output = np.zeros((dataset.shape[0], 1),dtype = int)
    for li, catg in enumerate(dataset[:,1]):
        output[li] = classToIndex(catg)
    return output        
    

In [ ]:
Y_number = setToNum(Y_raw_train)
print(Y_number.shape)
print(Y_number.dtype)

Y_train = keras.utils.to_categorical(Y_number, num_classes=classnum, dtype='int64')
print(Y_train.shape)

In [ ]:
"""建立模型"""
classes = [
#  '32PSK',
#  '16APSK',
#  '32QAM',
 'FM',
#  'GMSK',
#  '32APSK',
 'OQPSK',
#  '8ASK',
 'BPSK',
 '8PSK',
 'AM-SSB-SC',
 '4ASK',
 '16PSK',
#  '64APSK',
#  '128QAM',
#  '128APSK',
 'AM-DSB-SC',
#  'AM-SSB-WC',
#  '64QAM',
 'QPSK',
#  '256QAM',
#  'AM-DSB-WC',
 'OOK']
#  '16QAM']
data_format = 'channels_first'

def residual_stack(Xm,kennel_size,Seq,pool_size):
    #1*1 Conv Linear
    Xm = Conv2D(32, (1,1), padding='same', name=Seq+"_conv1", kernel_initializer='glorot_normal',data_format=data_format)(Xm)
    #Residual Unit 1
    Xm_shortcut = Xm
    Xm = Conv2D(32, kennel_size, padding='same',activation="relu",name=Seq+"_conv2", kernel_initializer='glorot_normal',data_format=data_format)(Xm)
    Xm = Conv2D(32, kennel_size, padding='same', name=Seq+"_conv3", kernel_initializer='glorot_normal',data_format=data_format)(Xm)
    Xm = layers.add([Xm,Xm_shortcut])
    Xm = Activation("relu")(Xm)
    #Residual Unit 2
    Xm_shortcut = Xm
    Xm = Conv2D(32, kennel_size, padding='same',activation="relu",name=Seq+"_conv4", kernel_initializer='glorot_normal',data_format=data_format)(Xm)
    X = Conv2D(32, kennel_size, padding='same', name=Seq+"_conv5", kernel_initializer='glorot_normal',data_format=data_format)(Xm)
    Xm = layers.add([Xm,Xm_shortcut])
    Xm = Activation("relu")(Xm)
    #MaxPooling
    Xm = MaxPooling2D(pool_size=pool_size, strides=pool_size, padding='valid', data_format=data_format)(Xm)
    return Xm


in_shp = (1024,2)#X_train.shape[1:]   #每个样本的维度[1024,2]
#input layer
Xm_input = Input(in_shp)
Xm = Reshape([1,1024,2], input_shape=in_shp)(Xm_input)
#Residual Srack
Xm = residual_stack(Xm,kennel_size=(3,3),Seq="ReStk0",pool_size=(2,2))   #shape:(512,1,32)
Xm = residual_stack(Xm,kennel_size=(3,2),Seq="ReStk1",pool_size=(2,1))   #shape:(256,1,32)
Xm = residual_stack(Xm,kennel_size=(3,2),Seq="ReStk2",pool_size=(2,1))   #shape:(128,1,32)
Xm = residual_stack(Xm,kennel_size=(3,2),Seq="ReStk3",pool_size=(2,1))   #shape:(64,1,32)
Xm = residual_stack(Xm,kennel_size=(3,2),Seq="ReStk4",pool_size=(2,1))   #shape:(32,1,32)
Xm = residual_stack(Xm,kennel_size=(3,2),Seq="ReStk5",pool_size=(2,1))   #shape:(16,1,32)

#############################################################################
#      多次尝试发现减少一层全连接层能使loss下降更快
#      将AlphaDropout设置为0.3似乎比0.5效果更好
#############################################################################
#Full Con 1
Xm = Flatten(data_format=data_format)(Xm)
Xm = Dense(128, activation='selu', kernel_initializer='glorot_normal', name="dense1")(Xm)
Xm = AlphaDropout(0.3)(Xm)
#Full Con 2
# Xm = Flatten(data_format=data_format)(Xm)
# Xm = Dense(128, activation='selu', kernel_initializer='glorot_normal', name="dense2")(Xm)
# Xm = AlphaDropout(0.5)(Xm)
#Full Con 2
Xm = Dense(len(classes), kernel_initializer='glorot_normal', name="dense3")(Xm)
#SoftMax
Xm = Activation('softmax')(Xm)
#Create Model
model = Model.Model(inputs=Xm_input,outputs=Xm)
adam = keras.optimizers.Adam(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=[keras.metrics.categorical_accuracy])
model.summary()

In [ ]:
"""训练模型"""
#############################################################################
#      当val_loss连续10次迭代不再减小或总迭代次数大于100时停止
#      将最小验证损失的模型保存
#############################################################################
print(tf.test.gpu_device_name())
# filepath = 'drive/RadioModulationRecognition/Models/ResNet_Model_72w.h5'
history = model.fit(X_train,
    Y_train,
    batch_size=1024,
    epochs=1000,
#     verbose=2,
#     validation_data=(X_test, Y_test),
    #validation_split = 0.3,
                    
#     callbacks = [
#         keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=True, mode='auto'),
#         keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='auto')
#     ]
)

# we re-load the best weights once training is finished
# model.load_weights(filepath)

In [ ]:
loss_list = history.history['loss']
accuracy_list = history.history['categorical_accuracy']
plt.plot(range(len(loss_list)),loss_list)
plt.plot(range(len(accuracy_list)),accuracy_list)
plt.show()

In [ ]:
index_class = {
 0:'FM',
 1:'OQPSK',
 2:'BPSK',
 3:'8PSK',
 4:'AM-SSB-SC',
 5:'4ASK',
 6:'16PSK',
 7:'AM-DSB-SC', 
 8:'QPSK', 
 9:'OOK',
}

# class to name
def classToName(cls):
    return index_class[cls]

def onehotToNum(dataset):
    output = np.array(np.argmax(dataset, axis = 1), ndmin = 2).T
    return output

In [ ]:
train_Y_hat = model.predict(X_train, batch_size=1024)
trainout = onehotToNum(train_Y_hat)
total = 30000
traincorrect = 0
traincorrect += (trainout == Y_number).sum().item()

print(train_Y_hat.shape)
print('Train Accuracy of the model over the 30000 train modulations: {} %'.format(100 * traincorrect / total)) 

In [ ]:
test_Y_hat = model.predict(X_test, batch_size=1024)
testout = onehotToNum(test_Y_hat)

predictionlist = testout.tolist()
names = []

for line in predictionlist:
    names.append(classToName(line[0]))
    
outputs = np.asarray(names, dtype = str)
print(outputs)
categories = pd.DataFrame(outputs, columns = ['Category'])       
categories.to_csv('categories.csv')